We got this python notebook from the following GitHub repo (https://github.com/subpath/Keras_music_gereration) as a way to test our baseline:

### Music gereration firth Keras and TensorFlow backend

Plan was simple:
1. Read midi file, convert it to matrix of features
2. Create simple model with Keras and LSTM to learn the pattern
3. Use subsample of initial midi file as a input for model to generate pure art
4. Save prediction from model to midi file
.
.
.
5. PROFIT

<i> For disclamer: I've been using my old Dell Laptop with no GPU support</i>

In [2]:
import mido
import os
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np

Using TensorFlow backend.
/Users/StephanieBrito/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


##### Read midi file

In [53]:
mid = MidiFile('Data/Bamboleo.mid')  
# midis = []
# print(os.listdir('Data'))
# for file in os.listdir('Data'):
#     if file.endswith(".mid"):
#         print(file)
#         midis.append(MidiFile('Data/' + file, clip=True))
    #files.append(file)

In [4]:
# mid = MidiFile('Data/abusowillieCol.mid', clip=True)

# for i, track in enumerate(mid.tracks):
#     print('Track {}: {}'.format(i, track.name))
#     for msg in track:
#         if msg.type == 'program_change':
#             print(msg)

##### Extract notes sequence

In [58]:
notes = []

def isPercussion(channel):
    return channel in range(8, 17) or channel in range(35, 82)

for msg in mid:
#         if msg.type == 'program_change':
#             print(msg)
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1])

# for file in os.listdir('Data')[1:6]:
#     mid = MidiFile('Data/' + file, clip=True)
#     for msg in mid:
# #         if msg.type == 'program_change':
# #             print(msg)
#         if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
#             data = msg.bytes()
#             notes.append(data[1])

#### Apply min-max scalling

In [61]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

/Users/StephanieBrito/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


#### Prepare features for training and data subsample for prediction

In [71]:
# LSTM layers requires that data must have a certain shape
# create list of lists fist
notes = [list(note) for note in notes]

# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 8
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
# save a seed to do prediction later
X_test = X[-300:]
X = X[:-300]
y = y[:-300]
print(len(notes))

4110


#### Made sequential model with several layers, use LSTM as it time dependent data

I also whant to save checkpoints

In [72]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer='rmsprop')
filepath="./Checkpoints/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', period=5)

# network_input = X
# model = Sequential()
# model.add(LSTM(
#         512,
#         input_shape=(network_input.shape[1], network_input.shape[2]),
#         return_sequences=True
#     ))
# model.add(Dropout(0.3))
# model.add(LSTM(512, return_sequences=True))
# model.add(Dropout(0.3))
# model.add(LSTM(512))
# model.add(Dense(256))
# model.add(Dropout(0.3))
# model.add(Dense(n_vocab))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#### Train your model.
It might take a while, I was waiting for 1 hour with just 5 epoch

In [73]:
from keras.utils import to_categorical

ll = model.fit(np.array(X), np.array(y), 32, 50, verbose=1)

Epoch 1/50
3802/3802 [==============================] - 12s 3ms/step - loss: 0.0540
Epoch 2/50
3802/3802 [==============================] - 7s 2ms/step - loss: 0.0437
Epoch 3/50
3802/3802 [==============================] - 6s 2ms/step - loss: 0.0425
Epoch 4/50
3802/3802 [==============================] - 6s 2ms/step - loss: 0.0416
Epoch 5/50
3802/3802 [==============================] - 7s 2ms/step - loss: 0.0408
Epoch 6/50
3802/3802 [==============================] - 7s 2ms/step - loss: 0.0407
Epoch 7/50
3802/3802 [==============================] - 8s 2ms/step - loss: 0.0404
Epoch 8/50
3802/3802 [==============================] - 9s 2ms/step - loss: 0.0398
Epoch 9/50
3802/3802 [==============================] - 7s 2ms/step - loss: 0.0398
Epoch 10/50
3802/3802 [==============================] - 7s 2ms/step - loss: 0.0396
Epoch 11/50
3802/3802 [==============================] - 8s 2ms/step - loss: 0.0395
Epoch 12/50
3802/3802 [==============================] - 7s 2ms/step - loss: 0.0394


#### Make a prediction

In [74]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]


#### Save your result to new midi file

In [75]:
mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('50_epoch_8_note_seq_MSE.mid')

#### Just listen to it. The result is surreal!